Import module

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.svm import SVC
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
from nltk.tokenize import word_tokenize
import re
import warnings 

stop=set(stopwords.words('english'))
warnings.filterwarnings("ignore")
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load Data

In [38]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print('Train Data size :{}'.format(train_df.shape))
print('Test Data size :{}'.format(test_df.shape))
Merge_df = train_df.append(test_df,ignore_index=True)

Train Data size :(7613, 5)
Test Data size :(3263, 4)


Remove twitter handle, URL, http tags, punctuation, special characters, numbers

In [40]:
def remove_pattern(input_txt, pattern):
    reg_obj = re.compile(pattern)
    output_txt = reg_obj.sub(r'', input_txt)

    return output_txt   


Merge_df['text'] = Merge_df['text'].apply(lambda x: remove_pattern(x,"@[\w]*"))
Merge_df['text'] = Merge_df['text'].apply(lambda x: remove_pattern(x,'https?://\S+|www\.\S+'))
Merge_df['text'] = Merge_df['text'].apply(lambda x: remove_pattern(x,'<.*?>'))
Merge_df['text'] = Merge_df['text'].apply(lambda x: remove_pattern(x,"[^a-zA-Z# ]"))

Remove Stop words

In [41]:
def remove_stop_words(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop] 
    filtered_tweet = ' '.join(filtered_sentence)
    
    return filtered_tweet


Merge_df['text'] = Merge_df['text'].apply(lambda x: remove_stop_words(x))

Tokenize & Stemming

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


def tokenize_stem(text):
    token_words = word_tokenize(text)
    stem_words =[]
    for i in token_words:
        word = lemmatizer.lemmatize(i)
        stem_words.append(word)
        
    final_tweet = ' '.join(stem_words)
    
    return final_tweet


Merge_df['text'] = Merge_df['text'].apply(lambda x: tokenize_stem(x))

Output cleaned data

In [53]:
training_df = Merge_df[:7613]
testing_df = Merge_df[7613:]

training_df.to_csv("train_clean.csv")
testing_df.to_csv("test_clean.csv")

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=300, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(Merge_df['text'])

train_data  = tfidf[:7613]
test_data = tfidf[7613:]
print(tfidf.shape)

(10876, 300)
